# Conjugate Gradient Method

Given the importance of the conjugate gradient methods in the solving of linear systems, I explore its derivation, intuition, and use. 

Much of the content is adapted from [An Introduction to the Conjugate Gradient Method Without the Agonizing Pain](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf). Much of the explanation is taken directly from Jonathan's work because his explanation makes this method very easy to understand. I have added some of my own changes to clarify a few points.

This notebook is an important precursor to the 'Preconditioner Series'.

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_triangular
from sklearn.datasets import make_spd_matrix

from plotly import __version__
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.figure_factory as ff

%matplotlib inline
%load_ext autoreload
%autoreload 2
init_notebook_mode(connected=True)
np.set_printoptions(suppress=True)

## Helper methods

In [2]:
def plot_iters(xx, yy, zz, sol):
    data = [go.Surface(
        x=xx,
        y=yy,
        z=zz,
        opacity=0.5)]
    
    z = np.array([(sol[i].T @ A @ sol[i]) / 2 - b.T @ sol[i] for i in range(len(sol))])
    trace = go.Scatter3d(
        x = sol[:,0],
        y = sol[:,1],
        z = z,
        marker = dict(
            size=6,
            colorscale='Viridis'),
        line=dict(
            color='#1f77b4',
            width=3))

    data.append(trace)
    
    return iplot(go.Figure(data=data))

CG methods are applicable to **symmetric positive definite** (SPD) systems.

Recall that for $A$ to be SPD:

$$ x^TAx>0\ \ \ \text{for every nonzero vector}\ \ x$$

For $Ax=b$ with $A$ being SPD, the goal is to find $x$ such that the following optimization problem is solved:

$$ \min_{x \in \mathbb{R}^n} f(x)\ \ \ \text{for}\ \ \ f(x)=\frac{1}{2}x^TAx - b^Tx+c $$

It can be shown that, for an $A$ that is SPD, $f(x)$ is minimized by the solution to $Ax=b$.

We can visualize the minimization problem in 3D with an interactive visualization of a simple sample problem:

$$ A=\begin{bmatrix} 3 & 2 \\ 2 & 6 \end{bmatrix}\ \ \ \ b=\begin{bmatrix} 2 \\ -8 \end{bmatrix}\ \ \ \ c=0 $$

The solution to this specific problem is $x=\begin{bmatrix} 2 \\ -2 \end{bmatrix}$ and is the minimum point of the surface described by $f(x)$.

In [3]:
A = np.array([[3, 2], [2, 6]])
b = np.array([2, -8])

xrange = yrange = np.arange(-10, 10)
xx, yy = np.meshgrid(xrange, yrange)
xy = np.dstack((xx, yy))
zz = np.array([[(xy[i,j,:].T @ A @ xy[i,j,:]) / 2 - b.T @ xy[i,j,:] 
      for j in range(xy.shape[1])] for i in range(xy.shape[0])])

data = [go.Surface(
    x=xx,
    y=yy,
    z=zz,
    opacity=0.8,
    contours=go.surface.Contours(
        z=go.surface.contours.Z(
          show=True,
          usecolormap=True,
          highlightcolor="#42f462",
          project=dict(z=True))))]

layout = go.Layout(
    scene = dict(
        xaxis_title = 'x1',
        yaxis_title = 'x2',
        zaxis_title = 'f(x)'))

iplot(go.Figure(data=data, layout=layout))

In 2D, the surface can be visualized using contour and quiver plots.

In [4]:
v, u = np.gradient(zz, 2, 2)
fig = ff.create_quiver(xx, yy, u, v,
                       scale=.05,
                       arrow_scale=.2,
                       name='quiver',
                       line=dict(width=1))

data = [go.Contour(
            x = xrange,
            y = yrange,
            z = zz,
            opacity = 0.5)]

fig.add_trace(data[0])
iplot(fig)

If $f(x)=\frac{1}{2}x^TAx - b^Tx+c$, then $f'(x)$ can be calculated using the gradient of $f(x)$ with respect to each of its independent variables:

$$ f'(x)=\frac{1}{2}A^Tx+\frac{1}{2}Ax-b=\frac{1}{2}(A^T+A)x-b $$

If $A$ is symmetric, the equation simplifies to:

$$ f'(x)=Ax-b $$

Setting $f'(x)=0$ for either equation and solving for $x$ provides the solution to the original system $Ax=b$. Therefore, a critical point of $f(x)$ is the solution to $Ax=b$.

**But how can this $x$ be guaranteed to be the solution?**

Consider some arbitrary point $p$. It can be shown that

$$ f(p) = f(x) + \frac{1}{2}(p-x)^TA(p-x) $$

Earlier we defined $A$ to be SPD meaning $x^TAx>0$ for any nonzero $x$. Therefore for any $p\neq x$, $f(p)>f(x)$. Thus, $x$ is the guaranteed to be the global minimum of $f$.

## Steepest descent method

The steepest descent method is a natural exploitation of the characteristics of the quadratic function we just explored. From an initial arbitrary guess $x_{(0)}$, the method steps in the direction of steepest descent until some convergence criterion is satisfied.

The direction is calculated as $-f'(x_{(i)})=b-Ax_{(i)}$. The negative is there because $f'(x_{(i)})$ points in the direction of steepest ascent, opposite of the direction we want to advance in.

Now, for a few terms that will be used:

**Error:** $e_{(i)}=x_{(i)}-x$

Difference between the solution and our current guess.

**Residual:** $r_{(i)}=b-Ax_{(i)}$

Difference between $b$ and the $b$ calculated from our current guess of $x$.

It should also be noted that $r_{(i)}=-f'(x_{(i)})$.

Using these new definitions, we can show that $r_{(i)}=-Ae_{(i)}$:

$$\begin{align}  
r_{(i)} &= b-Ax_{(i)} \\
&= Ax-Ax_{(i)} \\ 
&= A(x-x_{(i)}) \\
&= -Ae_{(i)}
\end{align}
$$

Now, $x_{(i)}$ has to be updated to provide a more accurate solution. We do this by shifting $x_{(i)}$ by some multiple, $\alpha$, of $r_{(i)}$ each iteration:

$$x_{(i+1)}=x_{(i)}+\alpha r_{(i)}$$

**Line search**

The value of $\alpha$ is chosen such that $f$ is minimized along the direction of $r_{(i)}$ in a procedure known as a line search.

Thinking back to calculus, we know that the minimum of a quadratic function with $f'' >0$ is where $f'=0$.

In the case of the line search this means $f$ is minimized for the $i^{th}$ iteration along the direction of the residual where $\dfrac{d}{d\alpha}f(x_{(i)})=0$. Expanding this derivative by the chain rule:

$$ \dfrac{d}{d\alpha}f(x_{(i)})=f'(x_{(i)})^T\dfrac{d}{d\alpha}x_{(i)}=f'(x_{(i)})^Tr_{(i-1)} $$

If we set $ \dfrac{d}{d\alpha}f(x_{(i)})=0$, then $f'(x_{(i)})^Tr_{(i-1)}=0$. This brings about the need for $f'(x_{(i)})=-r_{(i)}$ and $r_{(i-1)}$ to be orthogonal.

The formula for $\alpha_{(i)}$, then, is the following:

$$ \alpha_{(i)} = \frac{r_{(i)}^Tr_{(i)}}{r_{(i)}^TAr_{(i)}} $$

**Steepest descent algorithm**
With the formulas mentioned, the steepest descent can be implemented in three main steps:
1. Calculate the residual for the current iteration:

$$ r_{(i)}=b-Ax_{(i)} $$
2. Calculate $\alpha$ using the residual calculated in Step 1:

$$ \alpha_{(i)} = \frac{r_{(i)}^Tr_{(i)}}{r_{(i)}^TAr_{(i)}} $$
3. Find $x_{(i+1)}$ using the $r_{(i)}$ and $\alpha_{(i)}$ calculated:

$$ x_{(i+1)}=x_{(i)}+\alpha_{(i)} r_{(i)} $$

4. If convergence criteria not satisfied, repeat Steps 1-3, otherwise return $x$

In [5]:
def steepest(A, x0, b, tol):
    x = np.array([x0])
    r = b - A @ x0
    
    while (np.linalg.norm(r) > tol):
        a = (r.T @ r) / (r.T @ A @ r) 
        x = np.vstack((x, x[-1] + a * r))
        r = b - A @ x[-1]
    
    return x

We can use the simple function above to iteratively solve for the sample linear system.

In [6]:
A = np.array([[3, 2], [2, 6]])
b = np.array([2, -8])
x0 = np.array([-8, -5])
tol = 1e-5

sol = steepest(A, x0, b, tol); sol

array([[-8.        , -5.        ],
       [-2.52729694,  0.77674212],
       [ 0.78762681, -2.36371196],
       [ 1.45112266, -1.66335523],
       [ 1.85301513, -2.04409546],
       [ 1.93345558, -1.95918609],
       [ 1.98217995, -2.00534602],
       [ 1.99193233, -1.99505183],
       [ 1.99783954, -2.00064814],
       [ 1.9990219 , -1.9994001 ],
       [ 1.99973807, -2.00007858],
       [ 1.99988142, -1.99992727],
       [ 1.99996824, -2.00000953],
       [ 1.99998562, -1.99999118],
       [ 1.99999615, -2.00000115],
       [ 1.99999826, -1.99999893]])

By plotting $x_{(i)}$ for each iteration, we can see how each iteration shifts the solution in orthogonal directions.

In [7]:
plot_iters(xx, yy, zz, sol)

## Conjugate direction method

Though the steepest descent method is intuitive, it is not the most efficient. Looking at the path the iterates take, it seems like the method takes steps in the same direction every other iteration. The conjugate directions method, then, eliminates this repetitive stepping in the same direction by taking exactly one step in each search direction before reaching convergence.

The key is to make the search directions **A-orthogonal (conjugate)**, hence the conjugate direction method. For vectors $d_{(i)}$ and $d_{(j)}$ to be A-orthogonal:

$$ d_{(i)}^TAd_{(j)}=0 $$

In the case of the conjugate directions method, the search direction of the current iteration $d_{(i)}$ must be A-orthogonal to the resulting error vector $e_{(i+1)}=x_{(i+1)}-x$.

Stepping from $x_{(i)}$ to $x_{(i+1)}$ is the same as before: 

$$ x_{(i+1)}=x_{(i)}+\alpha_{(i)}d_{(i)} $$

As with the method of steepest descent, the line search from $x_{(i)}$ is trying to find the minimum along the given search direction, which means that $\dfrac{d}{d\alpha}f(x_{(i+1)})=0$.

Expanding the derivative above using the chain rule, we get:

$$\begin{align}
\dfrac{d}{d\alpha}f(x_{(i+1)}) &= 0 \\
f'(x_{(i+1)})^T\dfrac{d}{d\alpha}x_{(i+1)} &= 0 \\
f'(x_{(i+1)})^T\dfrac{d}{d\alpha}(x_{(i)}+\alpha_{(i)}d_{(i)}) &= 0 \\
-r^T_{(i+1)}d_{(i)} &= 0 \\
d_{(i)}^TAe_{(i+1)} &= 0
\end{align} $$

Hence $d_{(i)}$ is A-orthogonal to $e_{(i+1)}$. Note that the proof above uses relationships for $e_{(i)}$ and $r_{(i)}$ shown in the section above.

Deriving $\alpha$ for this method begins with $d_{(i)}^TAe_{(i+1)} = 0$:

$$\begin{align}
d_{(i)}^TAe_{(i+1)} &= 0 \\
d_{(i)}^TA(e_{(i)} + \alpha_{(i)}d_{(i)}) &= 0 \\
d_{(i)}^TAe_{(i)} + d_{(i)}^TA\alpha_{(i)}d_{(i)} &= 0 \\
d_{(i)}^TA\alpha_{(i)}d_{(i)} &= -d_{(i)}^TAe_{(i)} \\
\alpha_{(i)} &= -\dfrac{d_{(i)}^TAe_{(i)}}{d_{(i)}^TAd_{(i)}} \\
\alpha_{(i)} &= -\dfrac{d_{(i)}^T(-r_{(i)})}{d_{(i)}^TAd_{(i)}} \\
\alpha_{(i)} &= \dfrac{d_{(i)}^Tr_{(i)}}{d_{(i)}^TAd_{(i)}}
\end{align} $$

The result of the search directions being A-orthogonal is that the conjugate directions method converges in $n$ steps, where $n$ is the number of dimensions.

The only thing left to do is calculate the A-orthogonal search directions {$d_{(i)}$}. This is performed using the **conjugate Gram-Schmidt process**:

$$ d_{(i)}=u_i+\sum_{k=0}^{i-1}\beta_{ik}d_{(k)}\ \ \text{for}\ \ i>0 $$

where $u_0, u_1,\dots,u_{n-1}$ are $n$ linearly independent vectors, $d_{(0)}=u_0$, $\beta_{ij}=\dfrac{-u_i^TAd_{(j)}}{d_{(j)}^TAd_{(j)}}$

In [8]:
def find_beta(u, A, d):
    return -(u.T @ A @ d) / (d.T @ A @ d)

def conj_gs(A):
    n = A.shape[0]
    u = np.eye(n)
    D = [u[0]]
    
    for i in range(1, n):
        d = u[i] + np.sum([find_beta(u[i], A, D[k]) * D[k] for k in range(i)], axis=0)
        D = np.vstack((D, d))
    
    return D

def conj_dir(A, x0, b):
    D = conj_gs(A)
    x = np.array([x0])
    
    for i in range(A.shape[0]):
        a = (D[i].T @ (b - A @ x[-1])) / (D[i].T @ A @ D[i])
        x = np.vstack((x, x[-1] + a * D[i]))

    return x

As expected, the conjugate directions method converges in 2 steps for this system.

In [9]:
A = np.array([[3, 2], [2, 6]])
b = np.array([2, -8])
x0 = np.array([-8, -5])

sol = conj_dir(A, x0, b); sol

array([[-8., -5.],
       [ 4., -5.],
       [ 2., -2.]])

Let's take a look at how the solution was reached.

In [10]:
plot_iters(xx, yy, zz, sol)

The downside to the conjugate directions method is that it is computationally expensive to calculate the A-orthogonal search directions.

## Conjugate gradient method

Conjugate gradient (CG) is a slight modification of the method of conjugate directios where the search directions are constructed by conjugation of the residuals ($u_i=r_{(i)}$).

As Shewchuk points out, the choice to use residuals provides the following benefits:
1. The residual is orthogonal to previous search directions meaning that it always provides a new search direction unless the residual is zero (which means the problem has been solved).
2. 

The subspace span $\{r_{(0)},r_{(1)},\dots,r_{(i-1)}\}$ is equal to $D_i$. Each residual is orthogonal to all previous residuals:

$$ r_{(i)}^Tr_{(j)}=0,\ \ i\neq j $$

$$\begin{align}D_i &= \text{span}\{d_{(0)},Ad_{(0)},A^2d_{(0)},\dots,A^{i-1}d_{(0)}\} \\
&= \text{span}\{r_{(0)},Ar_{(0)},A^2r_{(0)},\dots,A^{i-1}r_{(0)}\}\end{align}$$

This subspace is known as a Krylov subspace.

In [ ]:
def conj_grad(A, x0, b):
    r = d = np.array([b - A @ x0])
    x = np.array([x0])
    
    for i in range(1, A.shape[0] + 1):
        a = (r[-1].T @ r[-1]) / (d[-1].T @ A @ d[-1])
        x = np.vstack((x, x[-1] + a * d[-1]))
        r = np.vstack((r, r[-1] - a * A @ d[-1]))
        beta = (r[-1].T @ r[-1]) / (r[-2].T @ r[-2])
        d = np.vstack((d, r[-1] + beta * d[-1]))
    
    return x

In [ ]:
A = np.array([[3, 2], [2, 6]])
b = np.array([2, -8])
x0 = np.array([-8, -5])

sol = conj_grad(A, x0, b); sol

In [ ]:
plot_iters(xx, yy, zz, sol)

## Preconditioned conjugate gradient method

## Resources

[Matrix Computations by Gene H. Golub and Charles F. Van Loan (4th Ed.)](https://www.amazon.com/Computations-Hopkins-Studies-Mathematical-Sciences/dp/1421407949)

[An Introduction to the Conjugate Gradient Method Without the Agonizing Pain](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf)